# $orvara$ tutorial

## Please make sure you are running this notebook from within the root directory!

In [ ]:
ls

### The above verifies that you are in the root directory.
You should see something like:


    build/                    HGCA_vEDR3.fits  orvara/           README.rst
    CHANGES.md                LICENCE          orvara.egg-info/  setup.py
    HGCA_vDR2_corrected.fits  misc/            pytest.ini        Tutorial.ipynb

## Now that you are in the root directory, lets install orvara.

### Warning about path names:
DO NOT run orbit fits from paths that contain files with spaces. This can cause python to fail at finding various input files. So if you install orvara and run from:

    /user/your name/Research Projects/orvara/
That may cause failures. But:

    /user/yourname/ResearchProjects/orvara/

is OK. If you are stuck with path names with spaces: You can always try first to see if it works with your directory paths, if you have spaces involved. But if you get a wierd error (especially when you the orbit fit has yet to actually start and is only trying to load in data), your path names containing spaces are probably the issue.

# Note: there is no need to run this following cell if you already have orvara installed.

In [ ]:
!pip install -e .

If you see: "Successfully installed orvara-X.X.X"

With X.X.X being the version number,
Great!

## if you installed orvara with the above cell, then restart the Kernal in this jupyter notebook, then continue with the next cell.

### Run pytest to make sure that the tests pass

This will run a small suite of tests. This should take < 30 seconds. It will complete with a few warnings -- that is OK.

**If this cell hangs for a long time (more than 3 minutes)-- please skip this cell and continue.**

If any of the tests fail, something is wrong with the install. Try running "pip install -e ." again If the issue persists, please submit an issue ticket!

In [ ]:
!pytest -sv -m "not e2e"

### Check the usage of fit_orbit and plot_orbit

fit_orbit and plot_orbit are the key commands to operate this package. Run --h to see their functionality. 

both commands require one optional argument --output-dir immediately followed by the desired directory to write output to. fit_orbit outputs the MCMC chain from the fit and plot_orbit outputs a set of publication-quality plots. If --output-dir flag is omitted, the output directory will be automatically set to the current working directory by default. 

Following the output directory is the configuration file tailored to the system you are fitting. Example configuration files can be found in /orvara/tests/.

In [ ]:
!fit_orbit --h

In [ ]:
!plot_orbit --h

### Running an example of fit_orbit, we will fit orbit to the White Dwarf - Main Sequence (WD-MS) binary system HD159062. 

The customizable configuration file for HD 159062 is located in orvara/tests/

So let's load the config.ini file for HD 159062 here:

In [ ]:
ls orvara/tests/

In [ ]:
def display_file(fpath):
    with open(fpath, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.rstrip('\n')
            print(line)

In [ ]:
print('This is what an orvara configuration file looks like:')
display_file('orvara/tests/config_HD159062.ini')

## Take note of the setting above in mcmc_settings:
    use_epoch_astrometry = True

This uses a precise treatment for the Hipparcos and Gaia absolute astrometry. If it is set to True, then the directory paths in the config file: GaiaDataDir, Hip1DataDir, and Hip2DataDir, have to exist. Orvara will not make them for you. Orvara will, however, download the appropriate Intermediate Data Files for your source into those folders. Note that for this tutorial the directories have already been created and the files have already been downloaded.

For a new fit, it is possible that downloading of the intermediate data files does not work. In this case ,you can just set use_epoch_astrometry = False and proceed with your fits. In most cases, setting it to False is entirely fine and will result in nearly identical fits to the True case. You can troubleshoot (or download the files manually and place them in the appropriate directories) and set this parameter to True later. 

Next, let's load the data sets needed for orbit fitting. There are three data types in general: RV, relative astrometry, or/and absolute astrometry. HD 159062 has all. Example data files are located in orvara/tests/data. Let's display them here:

In [ ]:
ls orvara/tests/data

This is the RV data, it has four columns: Epoch[JD], RV[m/s], RV_err[m/s] and InstrumentID

In [ ]:
print('This is what an orvara RV data input file looks like:')
display_file('orvara/tests/data/HD159062_RV.dat')

This is the relative astrometry file for HD 159062. It has seven columns: Date      , Sep (arcsec) , Err_sep  ,    PA (degrees),  Err_PA  Corr_Sep_PA, and  PlanetID. Planet ID depends on how many planets are in the system, 0 denotes the innermost planet, the second innermost is denoted 1 and so on.

In [ ]:
print('This is what an orvara relative astrometry data input file looks like:')
display_file('orvara/tests/data/HD159062_relAST.txt')

For absolute astrometry from the HGCA, make sure that the HGCA file (DR2 or eDR3) is specified. We are using Gaia eDR3 version of the HGCA.

To use intermediate astrometry, make sure that the parameter use_epoch_astrometry in the config file is set to True.

We first recommend running fits with use_epoch_astrometry set to False, then turning it on later.

Make sure that the directories for the Gaia and Hipparcos intermediate data, set in the .ini config file, are valid directories. You will need one directory each for Hip1, Hip2, and Gaia.

orvara will download any file you need automatically (and save them to the directories you specified), but:

You may download the files by hand if you want, and simply put them in the data directory, with filenames containing the hip ID. The Gaia eDR3 observations are publicly accessible via the Gaia Observation Forecast Tool (https : //gaia.esac.esa.int/gost/index.jsp). Gaia files are .csv files queried from GOST. The Hip1 and Hip2 data are (if not, soon) avaliable publically from ESA.

Here are those files, with filenames containing HIP85653 are the ones we want for HD 159062. Remember, orvara should download these files automatically in normal use.

In [ ]:
ls orvara/tests/data/gaia/

In [ ]:
ls orvara/tests/data/hip1/

In [ ]:
ls orvara/tests/data/hip2/

### Now, let's run fit_orbit and plot_orbit commands in the root directory

We are going to run an example fit to HD 159062. Verifty that the RV data, relative astrometry data and absolute astrometry data are loaded in correctly to ensure a suceesful fit. Progress bar shows the progress of MCMC. 

In [ ]:
!fit_orbit --output-dir orvara/tests/chains orvara/tests/config_HD159062.ini

Once you have the MCMC chain output by fit_orbit, feed it to McmcDataFile= in the config file. You can set a burn-in phase for the chain. After that, you are ready to go on to plot the orbits. plot_orbit will generate a suite of 7 plots relavant to RV and Astrometry, and a plot of the MCMC chain to breifly check convergence, and two files containting the beststep and posterior parameters. 

In [ ]:
!plot_orbit --output-dir orvara/tests/plots/ orvara/tests/config_HD159062.ini

### Great! Your plots are generated in the default or specified output directory.

You can see the whole sets of plots in the output directory.

In [ ]:
ls orvara/tests/plots/

## For instance, here is the visual orbit.

In [ ]:
from IPython.display import HTML
HTML('<iframe src=./orvara/tests/plots/astrometric_orbit_HD159062B.pdf width=700 height=350></iframe>')

## We can also inspect the posteriors and chisquareds from the fit:

In [ ]:
print('Here are the chisquared values of the model with respect to the data (for all the input data types):')
display_file('orvara/tests/plots/beststep_params_HD159062B.txt')

In [ ]:
print('Here are the posteriors with +- 1 sigma confidence intervals:')
display_file('orvara/tests/plots/posterior_params_HD159062B.txt')